In [4]:
from stock_trading_agent.data_ingestion.historical_data import HistoricalData

# Define the variables
ticker = "AAPL"  # Example ticker
start_date = "2023-03-21"  # Example start date
end_date = "2025-03-21"  # Example end date

# Create an instance of HistoricalData and get the historical data
historical_data = HistoricalData(ticker, start_date, end_date)
data = historical_data.get_historical_data()

# Display the data
print(data)


ImportError: cannot import name 'RESTClient' from 'polygon' (/Users/shukla03/Library/Caches/pypoetry/virtualenvs/stock-trading-agent-4e92X3xZ-py3.13/lib/python3.13/site-packages/polygon/__init__.py)

In [ ]:
import pandas as pd
from stock_trading_agent.enums.ticker import Ticker
from stock_trading_agent.data_ingestion.historical_data import HistoricalData
import os
import time

start_date = "2023-03-21"
end_date = "2025-03-21"

all_data = []

for ticker in Ticker:
    print(ticker)
    historical_data = HistoricalData(ticker.value, start_date, end_date)
    data = historical_data.get_historical_data()
    all_data.append(data)

combined_data = pd.concat(all_data)
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../data/raw"))
os.makedirs(base_dir, exist_ok=True) 
file_path = os.path.join(base_dir, "all_tickers_data.parquet")
combined_data.to_parquet(file_path, compression='snappy')

Ticker.Apple_Inc
Ticker.Microsoft_Corporation
Ticker.Amazon_com_Inc
Ticker.Facebook_Inc
Ticker.Alphabet_Inc_Class_C
Waiting for ~ 1 minute to respect API rate limit...


In [ ]:
import pandas as pd
import os
import time
from stock_trading_agent.enums.ticker import Ticker
from stock_trading_agent.data_ingestion.historical_data import HistoricalData
from polygon.exceptions import ResponseError

# Define start and end dates for the historical data
start_date = "2023-03-21"
end_date = "2025-03-21"

# This will hold all the data from each ticker
all_data = []

# Maximum number of retries if rate-limited
max_retries = 5
# Initial wait time (in seconds) for backoff
wait_time = 5

def fetch_data_with_retry(ticker, retries=max_retries, wait_time=wait_time):
    """Fetch historical data for a ticker with exponential backoff in case of rate limit errors."""
    for attempt in range(retries):
        try:
            # Initialize HistoricalData object
            historical_data = HistoricalData(ticker.value, start_date, end_date)
            # Fetch the data
            data = historical_data.get_historical_data()
            return data
        except ResponseError as e:
            if e.response.status_code == 429:  # Handle 429: Too Many Requests (rate limit)
                print(f"Rate limit hit for {ticker}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)  # Wait before retrying
                wait_time *= 2  # Exponential backoff (double the wait time)
            else:
                print(f"Unexpected error while fetching data for {ticker}: {e}")
                break  # Break if it's not a rate limit issue
    print(f"Max retries reached for {ticker}. Skipping.")
    return None  # Return None if max retries are reached

# Iterate over each ticker and fetch its data
for ticker in Ticker:
    print(f"Fetching data for {ticker}")
    data = fetch_data_with_retry(ticker)
    
    if data is not None:
        all_data.append(data)
    else:
        print(f"Skipping {ticker} due to failure to fetch data.")

# Combine all data into a single DataFrame
if all_data:
    combined_data = pd.concat(all_data)
else:
    print("No data was fetched. Exiting.")
    exit()

# Save the combined data to a Parquet file
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../data/raw"))
os.makedirs(base_dir, exist_ok=True)  # Create directory if it doesn't exist
file_path = os.path.join(base_dir, "all_tickers_data.parquet")
combined_data.to_parquet(file_path, compression='snappy')

print(f"Data saved to {file_path}")


In [11]:
import pandas as pd

# Define the file path
file_path = '/Users/shukla03/Developer/MLLearning/stock-trading-agent/data/raw/all_tickers_data.parquet'

# Read the Parquet file
df = pd.read_parquet(file_path, engine='pyarrow')  # or engine='fastparquet' if you prefer that

# Display the DataFrame
print(df.head())  # Display the first few rows of the DataFrame


     open    high      low   close      volume      vwap     timestamp  \
0  158.86  160.34  157.850  160.25  59006343.0  159.2417  1.679630e+12   
1  159.94  160.77  157.870  158.28  52390266.0  159.0188  1.679890e+12   
2  157.97  158.49  155.980  157.65  45992152.0  156.9785  1.679976e+12   
3  159.37  161.05  159.350  160.77  51305691.0  160.2160  1.680062e+12   
4  161.53  162.47  161.271  162.36  49210289.0  162.0209  1.680149e+12   

   transactions   otc ticker  
0      517531.0  None   AAPL  
1      518436.0  None   AAPL  
2      432447.0  None   AAPL  
3      472077.0  None   AAPL  
4      456152.0  None   AAPL  
